In [1]:
# Core YOLO
from ultralytics import YOLO  # main YOLOv8 library

# Basic utilities
import os
import shutil
import glob
import random

# Numerical + Data Handling
import numpy as np
import pandas as pd

# Visualization
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Progress bar
from tqdm import tqdm

# System checks
import torch
from warnings import filterwarnings
filterwarnings("ignore")

In [4]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))


PyTorch version: 2.8.0+cpu
CUDA available: False
